For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

1. Data cleaning / processing / language parsing
2. Create features using two different NLP methods: For example, BoW vs tf-idf.
3. Use the features to fit supervised learning models for each feature set to predict the category outcomes.
4. Assess your models using cross-validation and determine whether one model performed better.
5. Pick one of the models and try to increase accuracy by at least 5 percentage points.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
# This assignment will use Shakespeare's Hamlet and Whitman's Leaves

# Load and view the data
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
leaves = gutenberg.raw('whitman-leaves.txt')

#print(hamlet)
#print(leaves)

In [4]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text


# The Chapter indicator is idiosyncratic
hamlet = re.sub(r'Chapter \d+', '', hamlet)
leaves = re.sub(r'CHAPTER .*', '', leaves)
    
hamlet = text_cleaner(hamlet)
leaves = text_cleaner(leaves)

In [5]:
# See if the cleaners worked
#print(hamlet)
#print(leaves)

In [6]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
hamlet_doc = nlp(hamlet)
leaves_doc = nlp(leaves)

In [7]:
# Group into sentences.
hamlet_sents = [[sent, "Shakespeare"] for sent in hamlet_doc.sents]
leaves_sents = [[sent, "Whitman"] for sent in leaves_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(hamlet_sents + leaves_sents)
sentences.head()

,0,1
0,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare
1,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare
2,"(Barnardo, .)",Shakespeare
3,"(Who, 's, there, ?)",Shakespeare
4,"(Fran, .)",Shakespeare


In [8]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
hamletwords = bag_of_words(hamlet_doc)
leaveswords = bag_of_words(leaves_doc)

# Combine bags to create a set of unique words.
common_words = set(hamletwords + leaveswords)

In [9]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


,foolish,tie,meerely,inside,damn'd,axe,siluer'd,listen,plaid,outside,...,vnfortifi,not,wander,fortify,fill,deathless,passionate,pious,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Barnardo, .)",Shakespeare
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Who, 's, there, ?)",Shakespeare
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Fran, .)",Shakespeare


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

lr = LogisticRegression()

print(cross_val_score(lr, X, Y))

[ 0.90866632  0.93149974  0.88946549]


# Set up a model using tf-idf

In [11]:
sentences_list=[]
for index, row in sentences.iterrows():
    sen = str(row[0])
    sentences_list.append(sen)

print(sentences_list[0:4])

['Actus Primus. Scoena Prima.', 'Enter Barnardo and Francisco two Centinels.', 'Barnardo.', "Who's there?"]


In [12]:
# Using tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
sentences_tfidf=vectorizer.fit_transform(sentences_list)
print("Number of features: %d" % sentences_tfidf.get_shape()[1])

Number of features: 7360


In [13]:
lr = LogisticRegression()

print(cross_val_score(lr, sentences_tfidf, sentences[1]))

[ 0.90295797  0.9097042   0.86974572]


### Bag of Words Method:
[ 0.90866632  0.93149974  0.88946549]

### tf-idf Method:
[ 0.90295797  0.9097042   0.86974572]

### Take a model and try to increase its score by at least five points.

In [14]:
# Using BoW and adding more features to improve the cv score
# making more features that take advantage of the spaCy information 

def parts_of_speech(text):
    pos = [token.pos_ for token in text]
    return [item for item in pos]

def pos_features(sentences, common_pos):
    d2 = pd.DataFrame(columns=common_pos)
    d2['text_sentence'] = sentences[0]
    d2['text_source'] = sentences[1]
    d2.loc[:, common_pos] = 0
    
    for i, sentence in enumerate(d2['text_sentence']):
        parts = [token.pos_
                for token in sentence
                if (
                    token.pos_ in common_pos
                )]
        for part in parts:
            d2.loc[i, part] += 1
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return d2

# Set up the groups.
hamletpos = parts_of_speech(hamlet_doc)
leavespos = parts_of_speech(leaves_doc)

# Combine groups to create a set of parts of speech.
common_pos = set(hamletpos + leavespos)

In [15]:
pos_counts = pos_features(sentences, common_pos)
pos_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


,NUM,VERB,ADJ,PART,ADP,NOUN,DET,ADV,PRON,X,SYM,PUNCT,INTJ,CCONJ,PROPN,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,4,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare
1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,3,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,"(Barnardo, .)",Shakespeare
3,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,"(Who, 's, there, ?)",Shakespeare
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,"(Fran, .)",Shakespeare


In [16]:
frames = [word_counts, pos_counts]

combined = pd.concat(frames, axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
combined.head()

,foolish,tie,meerely,inside,damn'd,axe,siluer'd,listen,plaid,outside,...,ADV,PRON,X,SYM,PUNCT,INTJ,CCONJ,PROPN,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,4,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,3,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,"(Barnardo, .)",Shakespeare
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,"(Who, 's, there, ?)",Shakespeare
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,"(Fran, .)",Shakespeare


In [17]:
combined = combined.groupby(level=0, axis=1).last()

In [18]:
Y = combined['text_source']
X = np.array(combined.drop(['text_sentence','text_source'], 1))
print(cross_val_score(lr, X, Y))

[ 0.91126103  0.93201868  0.91489362]


In [19]:
d3 = pd.DataFrame()
d3['text_sentence'] = sentences[0]
d3['text_source'] = sentences[1]
d3.loc[:, 'word_counts'] = 0

for i, sentence in enumerate(d3['text_sentence']):
    word_counts = 0
    words = [token
             for token in sentence
             if (
                 not token.is_punct
                 and not token.is_stop
                 
                 )]
    word_counts = len(words)
    d3.loc[i, 'word_counts'] = word_counts
    if i % 500 == 0:
        print('Processing row {}'.format(i))
        
d3.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


,text_sentence,text_source,word_counts
0,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare,4
1,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare,4
2,"(Barnardo, .)",Shakespeare,1
3,"(Who, 's, there, ?)",Shakespeare,1
4,"(Fran, .)",Shakespeare,1


In [20]:
frames = [combined, d3]

combined2 = pd.concat(frames, axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
combined2.head()

,'em,'s,1,1.play,10,11,12,13,2,3,...,yield,yon,yond,yong,you'l,young,youth,text_sentence,text_source,word_counts
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Barnardo, .)",Shakespeare,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Who, 's, there, ?)",Shakespeare,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Fran, .)",Shakespeare,1


In [21]:
combined2 = combined2.groupby(level=0, axis=1).last()

In [22]:
Y = combined2['text_source']
X = np.array(combined2.drop(['text_sentence','text_source'], 1))

print(cross_val_score(lr, X, Y))

[ 0.9071095   0.93357551  0.91333679]


### Results:
BOW: [ 0.90866632  0.93149974  0.88946549]

BOW and POS: [ 0.91126103  0.93201868  0.91489362]

BOW, POS, and Word Count: [ 0.9071095   0.93357551  0.91333679]

### Using BOW and POS generated the highest scores.